<a href="https://colab.research.google.com/github/lauraluebbert/test_gget_alphafold/blob/dev/test_gget_alphafold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [gget alphafold](https://github.com/pachterlab/gget) test

___

Install and import gget:

In [1]:
# Install gget
!uv pip uninstall gget
!uv pip install "mysql-connector-python>=8.0.32" biopython
!uv pip install --no-cache-dir git+https://github.com/pachterlab/gget.git@main
import gget

Using Python 3.12.11 environment at: /usr
Uninstalled 1 package in 152ms
 - gget==0.29.3 (from git+https://github.com/pachterlab/gget.git@5dc47f3a8e1c70e18af50308406a9ac40c2a70bd)
Using Python 3.12.11 environment at: /usr
Audited 2 packages in 126ms
Using Python 3.12.11 environment at: /usr
Resolved 102 packages in 16.07s
Prepared 1 package in 9.37s
Installed 1 package in 1ms
 + gget==0.29.3 (from git+https://github.com/pachterlab/gget.git@ee15b01c6b2b99105a9b84771e8f26a46a07db91)


Install OpenMM:  

In [2]:
# Install OpenMM 8
!uv pip install "openmm>=8,<9"

# Create a lightweight 'simtk' shim that re-exports OpenMM 8 symbols
import os, site, pathlib, textwrap

site_pkgs = site.getsitepackages()[0]  # e.g., /usr/local/lib/python3.x/dist-packages
shim_root = pathlib.Path(site_pkgs) / "simtk"
(shim_root / "openmm" / "app").mkdir(parents=True, exist_ok=True)

# simtk/__init__.py
(shim_root / "__init__.py").write_text("")

# simtk/openmm/__init__.py -> re-export openmm 8 API
(shim_root / "openmm" / "__init__.py").write_text(textwrap.dedent("""\
    # Compatibility shim for legacy imports: `import simtk.openmm as openmm`
    from openmm import *  # re-export core openmm symbols
    from openmm import unit as unit  # provide `simtk.openmm.unit` attribute
    try:
        # make `from simtk.openmm import app` work via attribute
        from openmm import app as app
    except Exception:
        pass
"""))

# simtk/openmm/app/__init__.py -> re-export app layer
(shim_root / "openmm" / "app" / "__init__.py").write_text(textwrap.dedent("""\
    # Compatibility shim for: `from simtk.openmm.app import *`
    from openmm.app import *
"""))

# simtk/unit.py -> legacy `from simtk import unit`
(shim_root / "unit.py").write_text("from openmm.unit import *\n")

# 3) Test the legacy imports
try:
    import simtk.openmm as openmm
    from simtk.openmm import app
    from simtk import unit
    print("Shim OK. OpenMM version:", openmm.version.version)
    # quick smoke test
    print("Has app.PDBFile:", hasattr(app, "PDBFile"))
    print("Has unit.nanometer:", hasattr(unit, "nanometer"))
except Exception as e:
    print("Shim/import failed:", repr(e))

# Install numpy and tensorflow versions compatible with Python 3.12
# Allow bigger wheels like TensorFlow to download (5 min timeout)
# !UV_HTTP_TIMEOUT=300 uv pip install --upgrade "numpy>=1.26,<2" "tensorflow-cpu>=2.17,<2.18"

Using Python 3.12.11 environment at: /usr
Audited 1 package in 139ms
Shim OK. OpenMM version: 8.3.1.dev-6e13f13
Has app.PDBFile: True
Has unit.nanometer: True


Install third-part dependencies and download AlphaFold model parameters using `gget setup` (this might take a few minutes):

In [3]:
gget.setup("alphafold")

INFO:gget.utils:Attempting to install py3Dmol using: uv pip install -U py3Dmol
INFO:gget.utils:py3Dmol installed successfully using uv pip install -U py3Dmol.
INFO:gget.utils:Installing AlphaFold from source (requires pip and git).
INFO:gget.utils:AlphaFold installed succesfully.
INFO:gget.utils:Installing pdbfixer from source (requires pip and git).
INFO:gget.utils:pdbfixer installed succesfully.
INFO:gget.utils:AlphaFold model parameters already downloaded.


Predict the 3D structure of CASP14 target [T1024](https://predictioncenter.org/casp14/target.cgi?id=8&view=all):

In [ ]:
gget.alphafold(
    "MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH",
    relax=True,
    out=None
    )

INFO:gget.utils:Validating input sequence(s).
INFO:gget.utils:Using the single-chain (monomer) model.
INFO:gget.utils:Finding closest source for reference database.
Jackhmmer search:   0%|          | 0/199 [elapsed: 00:00 remaining: ?]